In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [7]:
#path="C:\\Dropbox\\Google\\MSc BA\\Courses\\AML\\Kaggle\\Telstra\\Data\\"
path="C:\\Users\\stan\\Google Drive\\MSc BA\\Courses\\AML\\Kaggle\\Telstra\\Data\\"

trainFile="train.csv"
testFile="test.csv"
resourceFile="resource_type.csv"
eventTypeFile="event_type.csv"
logFeatureFile="log_feature.csv"
severityTypeFile="severity_type.csv"

test=pd.read_csv(filepath_or_buffer=path+testFile,delimiter=",",header=0)
train=pd.read_csv(filepath_or_buffer=path+trainFile,delimiter=",",header=0)
resource=pd.read_csv(filepath_or_buffer=path+resourceFile,delimiter=",",header=0)
event=pd.read_csv(filepath_or_buffer=path+eventTypeFile,delimiter=",",header=0)
feature=pd.read_csv(filepath_or_buffer=path+logFeatureFile,delimiter=",",header=0)
severity=pd.read_csv(filepath_or_buffer=path+severityTypeFile,delimiter=",",header=0)
join=pd.DataFrame({'id':[]})
features={'resource':resource,'event':event,'feature':feature,'severity':severity}

In [8]:
train['sample']='train'
test['sample']='test'
test['fault_severity']=np.nan
join=pd.concat([train,test],ignore_index=True)

fault_severity - response
severity_type - recorded system prediction of the failure, categorical
location - tied to the event together with a point in time - id
id - combined location and point in time event id
resource_type - categorical? - n:10 - test to see if there is a smooth transition in the response
event_type - categorical? - n:53 - test to see if there is smooth transition in the response
log_feature - categorical? - n:386
volume - continuous but int - 341

In [9]:
for key,dataset in features.items():
        join=pd.merge(join,dataset,on='id',how='outer')

join=join.set_index(['sample',join.index])

In [ ]:
pd.merge(join.loc['train'],join.loc['test'],on='location',how='outer')

In [5]:
import collections
cat=collections.namedtuple('CategoricalConverter',['map','numSeries'])
    
def catSeries2numSeries(catSeries):
    cats=np.sort(catSeries.unique())
    cat_mapping=dict(zip(cats,range(0,len(cats)+1)))
    numSeries=catSeries.map(cat_mapping).astype(int)
    obs=cat(cat_mapping,numSeries)
    return obs

catLabels=join.dtypes[join.dtypes.map(lambda x: x == 'object')]
catLabels=catLabels.index
#cat2int=dict(zip(catLabels,range(0,len(catLabels)+1)))
cat_map={}
counter=0
for label in catLabels:
    convert=catSeries2numSeries(join[label])
    cat_map[label]=convert.map
    join[label]=convert.numSeries
    #setting categorical mapping
join.head()

fault_severity     id  location  log_feature  volume  severity_type  \
sample                                                                          
train  0               1  14121       148          237      19              1   
       1               1  14121       148          237      19              1   
       2               1  14121       148          148      19              1   
       3               1  14121       148          148      19              1   
       4               0   9320      1027          240     200              1   

          resource_type  event_type  
sample                               
train  0              2          26  
       1              2          27  
       2              2          26  
       3              2          27  
       4              2          26

In [6]:
join=pd.get_dummies(join,columns=['location','severity_type','event_type', 'resource_type'])
join=join.to_sparse(fill_value=0)
join.head()

AttributeError: 'SingleBlockManager' object has no attribute 'to_dense'

AttributeError: 'SingleBlockManager' object has no attribute 'to_dense'

In [15]:
train=join.loc['train']
test=join.loc['test']

MemoryError: 

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)

features=['location','log_feature','severity_type','event_type', 'resource_type','volume']
train_features = train[features]
#train_features = train[['location']]

train_target = train['fault_severity']

test_features = test[features]

# Fit the model to our training data
clf = clf.fit(train_features, train_target)
score = clf.score(train_features, train_target)
"Mean accuracy of Random Forest: {0}".format(score)

test_response=clf.predict(test_features)

In [10]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split

# Split 80-20 train vs test data
train_x, test_x, train_y, test_y = train_test_split(train_features, 
                                                    train_target, 
                                                    test_size=0.20, 
                                                    random_state=0)
print (train_features.shape, train_target.shape)
print (train_x.shape, train_y.shape)
print (test_x.shape, test_y.shape)


clf = clf.fit(train_x, train_y)
predict_y = clf.predict(test_x)

from sklearn.metrics import accuracy_score
print ("Accuracy = %.2f" % (accuracy_score(test_y, predict_y)))

(61839, 6) (61839,)
(49471, 6) (49471,)
(12368, 6) (12368,)
Accuracy = 0.77


In [119]:
(join.loc['train']).head()

,fault_severity,id,location,severity_type,event_type,resource_type,log_feature,volume
0,1,14121,location 118,severity_type 2,event_type 34,resource_type 2,feature 312,19
1,1,14121,location 118,severity_type 2,event_type 34,resource_type 2,feature 232,19
2,1,14121,location 118,severity_type 2,event_type 35,resource_type 2,feature 312,19
3,1,14121,location 118,severity_type 2,event_type 35,resource_type 2,feature 232,19
4,0,9320,location 91,severity_type 2,event_type 34,resource_type 2,feature 315,200


In [8]:
train.dtypes

fault_severity    float64
id                  int64
location            int32
severity_type       int32
resource_type       int32
event_type          int32
log_feature         int32
volume              int64
dtype: object

In [166]:
for cat in catLabels:
    print(train[train['log_feature'].isnull()])

Empty DataFrame
Columns: [fault_severity, id, location, severity_type, event_type, resource_type, log_feature, volume, object]
Index: []
Empty DataFrame
Columns: [fault_severity, id, location, severity_type, event_type, resource_type, log_feature, volume, object]
Index: []
Empty DataFrame
Columns: [fault_severity, id, location, severity_type, event_type, resource_type, log_feature, volume, object]
Index: []
Empty DataFrame
Columns: [fault_severity, id, location, severity_type, event_type, resource_type, log_feature, volume, object]
Index: []
Empty DataFrame
Columns: [fault_severity, id, location, severity_type, event_type, resource_type, log_feature, volume, object]
Index: []


In [ ]:
#retired bogus implementation
import collections
cat=collections.namedtuple('CategoricalConverter',['map','numSeries'])
    
def catSeries2numSeries(catSeries):
    cats=np.sort(catSeries.unique())
    cat_mapping=dict(zip(cats,range(0,len(cats)+1)))
    numSeries=catSeries.map(cat_mapping).astype(int)
    obs=cat(cat_mapping,numSeries)
    return obs

catLabels=join.dtypes[join.dtypes.map(lambda x: x == 'object')]
catLabels=catLabels.index
cat_map={}
counter=0
for label in catLabels:
    convert=catSeries2numSeries(join[label])
    cat_map[label]=convert.map #attribute from CategoricalConverter
    join[label]=convert.numSeries
    #setting categorical mapping; this is bogus, since vars are categorical